In [1]:
### SETUP ###

import arcpy
from arcgis import gis
import pandas as pd
from arcpy.sa import *

arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'BiodiversityMBRC.gdb'
gdb = arcpy.env.workspace

### FUNCTIONS ###

# spatial join function that allows you to input fields to join
def simpleSpatialJoin(fcA, fcB, fcOut, fieldsToJoin, joinType, keepType):
    # Get field mappings of input dataset
    mapA = arcpy.FieldMappings()
    mapA.addTable(fcA)
    # Get field mappings of join dataset
    mapB = arcpy.FieldMappings()
    mapB.addTable(fcB)
    # Get index of sensitivity score field and add to field map. Define merge rule.
    for field in fieldsToJoin:
        ind = mapB.findFieldMapIndex(field)
        fmap = mapB.getFieldMap(ind)
        fmap.mergeRule = 'First'
        mapA.addFieldMap(fmap)
    # Run spatial join
    if joinType == 'JOIN_ONE_TO_ONE':
        match = 'LARGEST_OVERLAP'
    else:
        match = 'INTERSECT'
    arcpy.analysis.SpatialJoin(fcA, fcB, fcOut, joinType, keepType, mapA, match, None, "")
    delFields = ["Join_Count", "TARGET_FID"]
    for field in delFields:
        arcpy.management.DeleteField(fcOut, field)

# same as above, but with a search radius
def simpleSpatialJoinNear(fcA, fcB, fcOut, fieldsToJoin, joinType, keepType, radius):
    # Get field mappings of input dataset
    mapA = arcpy.FieldMappings()
    mapA.addTable(fcA)
    # Get field mappings of join dataset
    mapB = arcpy.FieldMappings()
    mapB.addTable(fcB)
    # Get index of sensitivity score field and add to field map. Define merge rule.
    for field in fieldsToJoin:
        ind = mapB.findFieldMapIndex(field)
        fmap = mapB.getFieldMap(ind)
        fmap.mergeRule = 'First'
        mapA.addFieldMap(fmap)
    # Run spatial join
    if joinType == 'JOIN_ONE_TO_ONE':
        match = 'CLOSEST'
    else:
        match = 'INTERSECT'
    arcpy.analysis.SpatialJoin(fcA, fcB, fcOut, joinType, keepType, mapA, match, f"{radius} Meters", "")
    
def replaceNone(dataset, field):
    with arcpy.da.UpdateCursor(dataset, field) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0
                cursor.updateRow(row)
                

In [ ]:
# Join basemap layer attributes to conservation network map
# !! Large amount of features. Runtime ~100 minutes for 100m grid

# will need to create new grid with offset receiving areas included in ecological corridors.

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

arcpy.management.MakeFeatureLayer('grid_clip_250m', 'grid')
# core veg
simpleSpatialJoin('grid', 'Basemap/coreVegetationCategorised_clean', 'Processing/gridCore', ['vegetation_cover_type', 'current_re', 'percent', 'size_category', 'core_area_id'],'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# stepping stones
simpleSpatialJoin('gridCore', 'Basemap/steppingStonesVegetationCategorised_clean', 'Processing/gridStep', ['size_category', 'stepping_stone_id'],'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# ecological corridors
arcpy.management.MakeFeatureLayer('Basemap/ecologicalCorridorsCategorised', 'ecoCorridors')
simpleSpatialJoin('gridStep', 'ecoCorridors', 'Processing/gridCorr', ['feature_type', 'eco_corridor_id', 'corridor_width'],'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# remant REs
simpleSpatialJoin('Processing/gridCorr', 'Input/Biodiversity_status_of_remnant_regional_ecosystems', 'Processing/gridRe', ['RE1', 'RE2', 'RE3', 'RE4', 'RE5', "RE_LABEL"], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# restoration area
simpleSpatialJoin('Processing/gridRe', 'Basemap/restorationAreas', 'Processing/gridResto', ['restoration_area_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')

# convert to pandas dataframe for column manipulation
grid = pd.DataFrame.spatial.from_featureclass("Processing/gridResto")
print(grid.columns)
grid.rename(columns={'size_category': 'core_size_category',
                    'size_category_1': 'stepping_stone_size_category',
                    'feature_type': 'eco_corridor_feature',
                    'current_re': 'refined_re',
                    'percent': 'refined_re_pc'},inplace=True)
# specify columns to export
gridSub = grid[['vegetation_cover_type', 'refined_re', 'refined_re_pc', 'RE1', 'RE2', 'RE3', 'RE4', 'RE5', 'RE_LABEL', 'core_size_category', 'core_area_id', 'stepping_stone_size_category', 'stepping_stone_id',
                'eco_corridor_feature', 'eco_corridor_id', 'restoration_area_id', 'SHAPE']]
# export to feature class
gridSub.spatial.to_featureclass('BiodiversityMBRC.gdb/Network_grids/vegGrid')
arcpy.management.CalculateField("vegGrid", "grid_id", "!OBJECTID!", "PYTHON", "", "SHORT")

In [33]:
# fauna habitat and movement

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# !! will have to join grid to points. Then join points to grid.

# save FMI datasets to list
fmi = arcpy.ListFeatureClasses('FMI*', '', 'Input')
for thisFMI in fmi:
    if thisFMI == fmi[0]:
        simpleSpatialJoinNear("Network_grids/vegGrid", f'Input/{thisFMI}', f'Processing/grid_join_{thisFMI}', ['SITE', 'ROAD', 'LOCATION', 'E', 'N'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
        lastFMI = thisFMI
    else:
        simpleSpatialJoinNear(f"Processing/grid_join_{lastFMI}", f'Input/{thisFMI}', f'Processing/grid_join_{thisFMI}', ['SITE', 'ROAD', 'LOCATION', 'E', 'N'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
        lastFMI = thisFMI
    df = pd.DataFrame.spatial.from_featureclass(f'Processing/grid_join_{thisFMI}').drop(columns=['Join_Count', 'TARGET_FID'])
    df.rename(columns={'site': f'{thisFMI}_site',
                      'road': f'{thisFMI}_road',
                      'location': f'{thisFMI}_location',
                       'e': f'{thisFMI}_e',
                       'n': f'{thisFMI}_n'},inplace=True)
    df.spatial.to_featureclass(f"BiodiversityMBRC.gdb/Processing/grid_join_{thisFMI}")

In [63]:
# fauna habitat and movement - FMI point datasets
if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

fmi = arcpy.ListFeatureClasses('FMI*', '', 'Input')

arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/fmiGrid")
grid = pd.DataFrame.spatial.from_featureclass("fmiGrid")

for thisFMI in fmi:
    output = f"Processing/{thisFMI}_join_grid"
    simpleSpatialJoinNear(f"Input/{thisFMI}", "Processing/fmiGrid", output, ['grid_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
    df = pd.DataFrame.spatial.from_featureclass(output)
    # drop null grid ids
    df = df.dropna(subset = 'grid_id')
    # convert columns to strings
    df = df.astype(str)
    # concatenate e and n columns
    df['e_n'] = '(' + df['e'].astype(str) + ', ' + df['n'].astype(str) + ')'
    # group by grid id and summarise relevant attributes
    df = df.groupby(by = 'grid_id', as_index=False).agg(lambda x: '; '.join(set(x))).reset_index()
    # convert grid id back to int
    df['grid_id'] = df['grid_id'].astype(int)
    # join columns to grid dataset
    grid = pd.merge(grid, df[['site', 'road', 'location', 'e_n', 'grid_id']], on='grid_id', how='left')
    grid.rename(columns={'site': f'{thisFMI}_site',
                      'road': f'{thisFMI}_road',
                      'location': f'{thisFMI}_location',
                       'e_n': f'{thisFMI}_e_n'},inplace=True)

grid.spatial.to_featureclass(f"BiodiversityMBRC.gdb/Processing/fmiGrid")

'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Processing\\fmiGrid'

In [9]:
# fauna habitat and movement - spatial join habitats - MUST RUN PREVIOUS CHUNK

arcpy.management.MakeFeatureLayer("Processing/fmiGrid", "fmiGrid")

# merge koala habitat
try:
    arcpy.management.Merge("Input/koala_habitat_areas_v4_0;Input/locally_refined_koala_habitat_areas_v4_0", 'Processing/Koala_habitat')
except:
    arcpy.management.Delete("Processing/Koala_habitat")
    arcpy.management.Merge("Input/koala_habitat_areas_v4_0;Input/locally_refined_koala_habitat_areas_v4_0", 'Processing/Koala_habitat')
    
# spatial join koala
try:
    simpleSpatialJoin("fmiGrid", "Koala_habitat", "Processing/fmiKoala", ["Dataset", "KPA", "HSM_CATEGORY"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
except:
    arcpy.management.Delete("Processing/fmiKoala")
    simpleSpatialJoin("fmiGrid", "Koala_habitat", "Processing/fmiKoala", ["Dataset", "KPA", "HSM_CATEGORY"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
    
# rename join fields
alterFields = {"Dataset": "koala_dataset",
              "KPA": "koala_priority_area",
              "HSM_CATEGORY": "koala_habitat_suitability"}

for oldField, newField in alterFields.items():
    arcpy.management.AlterField("fmiKoala", oldField, newField, newField)
    
# spatial join platypus
arcpy.management.MakeFeatureLayer("Input/Platypus_Habitat", "Platypus_Habitat")
try:
    simpleSpatialJoin("fmiKoala", "Platypus_Habitat", "Processing/fmiPlat", ["habitat"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
except:
    arcpy.management.Delete("Processing/fmiPlat")
    simpleSpatialJoin("fmiKoala", "Platypus_Habitat", "Processing/fmiPlat", ["habitat"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
    
# rename join fields
alterFields = {"habitat": "platypus_habitat"}

for oldField, newField in alterFields.items():
    arcpy.management.AlterField("fmiPlat", oldField, newField, newField)

# spatial join turtle nesting areas
arcpy.management.MakeFeatureLayer("Input/Sea_turtle_nesting_areas", "turtles")
try:
    simpleSpatialJoin("fmiPlat", "turtles", "Network_grids/faunaMovementGrid", ["SIGNIFICANCE"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
except:
    arcpy.management.Delete("Network_grids/faunaMovementGrid")
    simpleSpatialJoin("fmiPlat", "turtles", "Network_grids/faunaMovementGrid", ["SIGNIFICANCE"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
    
# rename join fields
alterFields = {"SIGNIFICANCE": "sea_turtle_habitat_significance"}

for oldField, newField in alterFields.items():
    arcpy.management.AlterField("faunaMovementGrid", oldField, newField, newField) 
    
# add shorebird roosts
arcpy.management.SelectLayerByLocation("faunaMovementGrid", "INTERSECT", "Input/Shorebird_Roosts", None, "NEW_SELECTION")
arcpy.management.CalculateField("faunaMovementGrid", "shorebird_roosts", "'Present'", "PYTHON3")
arcpy.management.SelectLayerByAttribute("faunaMovementGrid", "CLEAR_SELECTION")

<Result 'faunaMovementGrid'>

In [ ]:
## LAND USE GRID
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'BiodiversityMBRC.gdb'
gdb = arcpy.env.workspace

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create land use grid, set extent
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/luGrid")
arcpy.env.extent = "Processing/luGrid"

# create merged council owned land dataset
if not arcpy.Exists('Input/councilOwnedLand'):
    arcpy.management.Merge('Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Civic;Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Rec_and_Foreshore;Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Sports', 
                           'Processing/LGIP_merged',  None, 'NO_SOURCE_INFO')

    # pull out council land from MBRC Parcels
    arcpy.management.MakeFeatureLayer('Input/MBRCParcels_Registered_Parcels', 'councilParcels', "COUNCIL_LAND IN ('Council Freehold', 'Council reserve', 'Council Leasee')")
    arcpy.analysis.PairwiseErase('councilParcels', 'Processing/LGIP_merged', 'Processing/councilParcels_Erase')
    arcpy.management.Merge('Processing/councilParcels_Erase;Processing/LGIP_merged', 'Input/councilOwnedLand', None, 'NO_SOURCE_INFO')

# join council owned land and state-owned land to grid
jDict = {
    'councilOwnedLand':{
        'colname': 'council_land',
        'fields': ['COUNCIL_LAND', 'LANDUSE', 'TENURE']
    },
    'StateLand_Cadastral_data_QLD_CADASTRE_DCDB': {
        'colname': 'state_land',
        'fields': ['TENURE']
        }
}

i = 0
for dset, attr in jDict.items():
    thisLayer = attr['colname']
    if i == 0:
        grid = 'luGrid'
    else:
        grid = output
    output = f'Processing/{dset}_grid'
    simpleSpatialJoin(grid, f'Input/{dset}', output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1
        
# join green infrastructure eco connections to grid
arcpy.management.MakeFeatureLayer("Input/GreenInfrastructure_Ecological_connections", "green_eco")
simpleSpatialJoin(output, "green_eco", "Processing/green_eco_grid", ["Name", "Type", "Class"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
# rename fields
rename = {"Name": "g_i_eco_connection_name",
         "Type": "g_i_eco_connection_type",
         "Class": "g_i_eco_connection_class"}
for old, new in rename.items():
    arcpy.management.AlterField("green_eco_grid", old, new, new)

# join green infrastructure waterways/drainage to grid
arcpy.management.MakeFeatureLayer("Input/GreenInfrastructure_Waterways_and_drainage_lines", "green_ww")
simpleSpatialJoin("green_eco_grid", "green_ww", "Processing/green_ww_grid", ["OVL2_CAT", "Name", "CATCHMENT", "STREAM_ID"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
# rename fields
rename = {"OVL2_CAT": "g_i_waterway_ovl2_cat",
         "Name": "g_i_waterway_name",
         "CATCHMENT": "g_i_waterway_catchment",
         "STREAM_ID": "g_i_waterway_stream_id"}
for old, new in rename.items():
    arcpy.management.AlterField("green_ww_grid", old, new, new)
    
# join green infrastructure managed forest to grid. This dataset has multiple overlapping polygons, therefore we will create a grid column presence/absence for each of the four types:
#  -Protected area
# - Managed forest
# - Green infrastructure
# - Reserves of conservation significance
arcpy.management.MakeFeatureLayer("Input/GreenInfrastructure_GI_and_Managed_Forest", "forest")
sets = {'Green Infrastructure': 'g_i_area',
       'Managed Forest': 'g_i_managed_forest',
       'Protected Areas': 'g_i_protected_areas',
       'Reserves of Conservation Significance': 'g_i_sig_conservation_reserves'}

for subcat, colname in sets.items():
    arcpy.management.SelectLayerByAttribute("forest", "NEW_SELECTION", f"SubCategory = '{subcat}'")
    arcpy.management.SelectLayerByLocation("green_ww_grid", "INTERSECT", "forest", "", "NEW_SELECTION")
    arcpy.management.CalculateField("green_ww_grid", colname, "'Present'", "PYTHON3")

arcpy.management.SelectLayerByAttribute("green_ww_grid", "CLEAR_SELECTION")

# join local precincts to grid
arcpy.management.MakeFeatureLayer("Input/PlanningScheme_Local_Plan_Precincts", "precincts")
simpleSpatialJoin("green_ww_grid", "precincts", "Processing/precincts_grid", ["LP", "LP_PREC"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
rename = {"LP": "local_plan",
         "LP_PREC": "local_plan_prec"}
for old, new in rename.items():
    arcpy.management.AlterField("precincts_grid", old, new, new)
    
    
# join planning scheme datasets to grid
planDict = {
    'PlanningSchemeEAs_Environmental_areas': {
        'colname': 'plan_scheme_ea',
        'fields': ['OVL2_DESC']
    },
    'PlanningScheme_Zones': {
        'colname': 'plan_scheme_zones',
        'fields': ['LP', 'LP_PREC', 'ZONE_PREC', 'LVL1_ZONE']
    },
    'PlanningScheme_Priority_Development_Area': {
        'colname': 'plan_scheme_priority_dev_area',
        'fields': ['pda']
    },
    'PlanningSchemeEAs_Caboolture_West_Local_Plan': {
        'colname': 'plan_scheme_local_plan',
        'fields': ['LP']
    },
    'PlanningSchemeEAs_Waterways': {
        'colname': 'plan_scheme_waterways',
        'fields': ['CAT_DESC', 'OVL2_DESC', 'Name', 'CATCHMENT', 'buffer_m']
    },
    'PlanningScheme_North_Lakes_Locality': {
        'colname': 'plan_scheme_north_lakes_locality',
        'fields': ['Mask']
    },
    'PlanningScheme_Strategic_Framework_Place_Types': {
        'colname': 'plan_scheme_strategic_fw',
        'fields': ['PLACETYPE']
    },
    'MBRC_Planning_Scheme___Riparian_Wetland_Setbacks_Overlay_Waterways': {
        'colname': 'plan_scheme_r_w',
        'fields': ['Name', 'Dev_SetBac', 'Env_Buffer', 'STREAM_ID', 'STREAM_NO']
    },
    'MBRC_Planning_Scheme___Riparian_Wetland_Setbacks_Overlay_areas': {
        'colname': 'plan_scheme_setbacks_area',
        'fields': ['OVL2_CAT']
    }
}

i = 0
# join planning scheme datasets to grid
for dset, attr in planDict.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = 'precincts_grid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1


# join regional land use category to grid
arcpy.management.MakeFeatureLayer("Input/Regional_land_use_categories___South_East_Queensland_Regional_Plan_2023___ShapingSEQ", "rluc")
simpleSpatialJoin(output, "rluc", "Processing/rluc_grid", ["RLUC2023"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField("rlu_grid", "RLUC2023", "shaping_seq_rluc_2023", "shaping_seq_rluc_2023")

# join development areas to grid
arcpy.management.MakeFeatureLayer("Input/Development_area___South_East_Queensland_Regional_Plan_2023___ShapingSEQ", "dev")
simpleSpatialJoin("rluc_grid", "dev", "Processing/dev_grid", ["NAME"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField("dev_grid", "NAME", "shaping_seq_develop_area", "shaping_seq_develop_area")

# add MSES datasets to grid
# create dictionary to run for loop to add all MSES datasets to grid. Includes original dataset name, update column prefix and fields to join.
mses = {
    'MSES___High_ecological_value_waters___watercourse': {
        'colname': 'mses_hev_watercourse',
        'fields': ['MI_TYPE']
    },
    'MSES___High_ecological_value_waters___wetlands': {
        'colname': 'mses_hev_wetlands',
        'fields': ['MI_TYPE']
    },
    'MSES___High_ecological_significance_wetlands': {
        'colname': 'mses_hes_wetlands',
        'fields': ['SIGNIFICANCE']
    },
    'MSES___Regulated_vegetation___essential_habitat': {
        'colname': 'mses_rv_ehab',
        'fields': ['EHAB', 'REGROWTH', 'RVM_CAT']
    },
    'MSES___Protected_area___nature_refuges': {
        'colname': 'mses_pa_nat_refuges',
        'fields': ['NAME', 'PLAN_NUM', 'PA_NUM', 'ALT_NAME']
    },
    'MSES___Regulated_vegetation___category_C_endangered_or_of_concern': {
        'colname': 'mses_rv_cat_c_eoc',
        'fields': ['RE', 'PC_LABEL', 'VM_CLASS', 'VERSION', 'SOURCE', 'VM_STATUS']
    },
    'MSES___Regulated_vegetation___category_B_endangered_or_of_concern': {
        'colname': 'mses_rv_cat_b_eoc',
        'fields': ['RE', 'PC_LABEL', 'VM_CLASS', 'VERSION', 'SOURCE', 'VM_STATUS']
    },
    'MSES___wildlife_habitat___special_least_concern_animal': {
        'colname': 'mses_hab_slc',
        'fields': ['MSES_SLC_SBH', 'MSES_SLC_REC']
    },
    'MSES___wildlife_habitat___endangered_or_vulnerable_wildlife': {
        'colname': 'mses_hab_cev',
        'fields': ['MSES_CEV_HSM', 'MSES_CEV_SBH', 'MSES_CEV_REC']
    },
    'MSES___Legally_secured_offset_area___vegetation_offsets': {
        'colname': 'mses_veg_offset_area',
        'fields': ['PMAV_NO']
    }
}

i = 0
# run for loop to join all MSES datasets to grid
for dset, attr in mses.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = 'dev_grid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        if field == 'PC_LABEL':
            arcpy.management.AlterField(output, field, f'{thisLayer}_PERCENT', f'{thisLayer}_PERCENT')
        else:
            arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1
    
# add threatened ecological communities to grid
arcpy.management.JoinField(output, 're1', 'MNES_threatened_ecological_communities', 'Regional_ecosystem', ['MNES_community', 'Status', 'Comments__limits'])
rename = {'MNES_community': 'mnes_eco_community',
         'Status': 'mnes_eco_status',
         'Comments__limits': 'mnes_eco_comments'}
for old, new in rename.items():
    arcpy.AlterField(output, old, new, new)

# export land use grid
try:
    arcpy.management.CopyFeatures(output, "Network_grids/landUseGrid")
except:
    arcpy.management.Delete("Network_grids/landUseGrid")
    arcpy.management.CopyFeatures(output, "Network_grids/landUseGrid")
    
print('Land use grid created successfully')

In [10]:
# add MSES datasets to grid

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)


# create mses grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/msesWIPGrid")
# create dictionary to run for loop to add all MSES datasets to grid. Includes original dataset name, update column prefix and fields to join.
mses = {
    'MSES___High_ecological_value_waters___watercourse': {
        'colname': 'mses_hev_watercourse',
        'fields': ['MI_TYPE']
    },
    'MSES___High_ecological_value_waters___wetlands': {
        'colname': 'mses_hev_wetlands',
        'fields': ['MI_TYPE']
    },
    'MSES___High_ecological_significance_wetlands': {
        'colname': 'mses_hes_wetlands',
        'fields': ['SIGNIFICANCE']
    },
    'MSES___Regulated_vegetation___essential_habitat': {
        'colname': 'mses_rv_ehab',
        'fields': ['EHAB', 'REGROWTH', 'RVM_CAT']
    },
    'MSES___Protected_area___nature_refuges': {
        'colname': 'mses_pa_nat_refuges',
        'fields': ['NAME', 'PLAN_NUM', 'PA_NUM', 'ALT_NAME']
    },
    'MSES___Regulated_vegetation___category_C_endangered_or_of_concern': {
        'colname': 'mses_rv_cat_c_eoc',
        'fields': ['RE', 'PC_LABEL', 'VM_CLASS', 'VERSION', 'SOURCE', 'VM_STATUS']
    },
    'MSES___Regulated_vegetation___category_B_endangered_or_of_concern': {
        'colname': 'mses_rv_cat_b_eoc',
        'fields': ['RE', 'PC_LABEL', 'VM_CLASS', 'VERSION', 'SOURCE', 'VM_STATUS']
    },
    'MSES___wildlife_habitat___special_least_concern_animal': {
        'colname': 'mses_hab_slc',
        'fields': ['MSES_SLC_SBH', 'MSES_SLC_REC']
    },
    'MSES___wildlife_habitat___endangered_or_vulnerable_wildlife': {
        'colname': 'mses_hab_cev',
        'fields': ['MSES_CEV_HSM', 'MSES_CEV_SBH', 'MSES_CEV_REC']
    },
    'MSES___Legally_secured_offset_area___vegetation_offsets': {
        'colname': 'mses_veg_offset_area',
        'fields': ['PMAV_NO']
    }
}

i = 0
# run for loop to join all datasets to grid
for dset, attr in mses.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = 'msesWIPGrid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        if field == 'PC_LABEL':
            arcpy.management.AlterField(output, field, f'{thisLayer}_PERCENT', f'{thisLayer}_PERCENT')
        else:
            arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1

# join MBRC planning 

# export final grid
try:
    arcpy.management.CopyFeatures(output, 'Network_grids/msesGrid')
except:
    arcpy.management.Delete('Network_grids/msesGrid')
    arcpy.management.CopyFeatures(output, 'Network_grids/msesGrid')


In [17]:
## GDE grid

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)


# create gde grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/gdeWIPGrid")

# get gde feature classes
gdes = {
    'QLD_GDE_SURFACE_POINTS': 'gde_point',
    'QLD_GDE_SURFACE_LINES': 'gde_line',
    'QLD_GDE_POTENTIAL_AQUIFER': 'gde_pot_aquifer',
    'QLD_GDE_TERRESTRIAL': 'gde_terrestrial',
    'QLD_GDE_SURFACE_AREAS': 'gde_surface_area'
}

i = 0
# add gde layers to grid
for dset, layer in gdes.items():
    arcpy.management.MakeFeatureLayer(f'Input/{dset}', layer)
    if dset == 'QLD_GDE_SURFACE_POINTS':
        fields = ["GDE_CLASS", "RULE_ID", "RULE_NAME", "RULE_PART", "VENT_ID", "NAME", "ELEVATION", "LATITUDE", "LONGITUDE"]
    elif dset == 'QLD_GDE_POTENTIAL_AQUIFER':
        fields = ["GDE_CLASS", "RULE_ID", "RULE_NAME", "AQ_NAME", "GW_SALINTY", "GW_PH", "GW_RECHARG"]
    else:
        fields = ["GDE_CLASS", "RULE_ID", "RULE_NAME", "RULE_PART"]
    if i == 0:
        thisGrid = "gdeWIPGrid"
    else:
        thisGrid = output
    output = f"Processing/{layer}_grid"
    simpleSpatialJoin(thisGrid, layer, output, fields, "JOIN_ONE_TO_ONE", "KEEP_ALL")
    for field in fields:
        arcpy.management.AlterField(output, field, f"{layer}_{field}", f"{layer}_{field}")
    i += 1

try:
    arcpy.management.CopyFeatures(output, "Network_grids/gdeGrid")
except:
    arcpy.management.Delete("Network_grids/gdeGrid")
    arcpy.management.CopyFeatures(output, "Network_grids/gdeGrid")

In [8]:
## coastal and hydro features grid

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create WIP coastal hydro grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/hydroWIPGrid")

arcpy.env.extent = "hydroWIPGrid"

# add coastal landform and ripiarian landform attributes
arcpy.management.SelectLayerByAttribute("hydroWIPGrid", "NEW_SELECTION", "current_re LIKE '%12.1.5%'")
arcpy.management.CalculateField("hydroWIPGrid", "coastal_landform", "'Yes'", 'PYTHON3')

arcpy.management.SelectLayerByAttribute("hydroWIPGrid", "NEW_SELECTION", "current_re LIKE '%12.3.5%'")
arcpy.management.CalculateField("hydroWIPGrid", "riparian_floodplain_landform", "'Yes'", 'PYTHON3')

# add coastal management districts to grid
arcpy.management.MakeFeatureLayer('Input/Coastal_plan_coastal_management_district', 'cmd')
arcpy.management.SelectLayerByLocation("hydroWIPGrid", "INTERSECT", "cmd", None, "NEW_SELECTION")
arcpy.management.CalculateField("hydroWIPGrid", "coastal_management_district", "'Yes'", 'PYTHON3')
arcpy.management.SelectLayerByAttribute("hydroWIPGrid", "CLEAR_SELECTION")

# create barrier works category
arcpy.management.MakeFeatureLayer("Input/Queensland_waterways_for_waterway_barrier_works", "barrier")
arcpy.management.CalculateField("barrier", "category", "''", "PYTHON3")
with arcpy.da.UpdateCursor("barrier", ["FISH_PASS", "category"]) as cursor:
    for row in cursor:
        if row[0] == 5:
            row[1] = "Major (Tidal)"
        elif row[0] == 4:
            row[1] = "Major"
        elif row[0] == 3:
            row[1] = "High"
        elif row[0] == 2:
            row[1] = "Moderate"
        elif row[0] == 1:
            row[1] = "Low"
        else:
            row[1] = "N/A"
        cursor.updateRow(row)

# create dictionary layers/fields map dictionary
hydro = {
    'Watercourse_identification_map___downstream_limits': {
        'colname': 'downstream_limit',
        'fields': ['NAME', 'OFFICE', 'REFERENCE', 'DETERMINED', 'CERTIFIED', 'URL'] 
    },
    'Water_storage_points': {
        'colname': 'w_storage_point',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Waterpoints': {
        'colname': 'w_point',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Waterholes': {
        'colname': 'waterhole',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Waterfalls': {
        'colname': 'waterfall',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Queensland_waterways_for_waterway_barrier_works': {
        'colname': 'ww_barrier_works',
        'fields': ['category']
    },
    'Major_watercourse_lines': {
        'colname': 'major_watercourse',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Watercourse_lines_NE_Coast_drainage_division___southern_section': {
        'colname': 'watercourse',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Canal_lines': {
        'colname': 'canal_line',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Pondage': {
        'colname': 'pondage',
        'fields': ['FEATURETYPE']
    },
    'Reservoirs': {
        'colname': 'reservoir',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Watercourse_areas': {
        'colname': 'watercourse_area',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Lakes': {
        'colname': 'lake',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Flats': {
        'colname': 'flat',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Canal_areas': {
        'colname': 'canal_area',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'MBRC_PlanningScheme_OM_Stormwater_Catchments': {
        'colname': 'stormw_catchment',
        'fields': ['OVL2_DESC']
    }
}

# join attributes to grid using dictionary
i = 0
for dset, attr in hydro.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = 'hydroWIPGrid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1

# copy output grid to grid FD
try:
    arcpy.management.CopyFeatures(output, "Network_grids/coastalHydroGrid")
except:
    arcpy.management.Delete("Network_grids/coastalHydroGrid")
    arcpy.management.CopyFeatures(output, "Network_grids/coastalHydroGrid")


In [10]:
## SPECIES OBSERVATION

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create WIP species grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/speciesWIPGrid")

grid = pd.DataFrame.spatial.from_featureclass("speciesWIPGrid")

dsets = ['ALA_records_20240103', 'Wildnet_equalto_prioritysp']

for dset in dsets:
    output = f"Processing/{dset}_join_grid"
    simpleSpatialJoinNear(f"Input/{dset}", "Processing/speciesWIPGrid", output, ['grid_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
    df = pd.DataFrame.spatial.from_featureclass(output)
    # drop null grid ids
    df = df.dropna(subset = 'grid_id')
    # convert columns to strings
    df = df.astype(str)
    # group by grid id and summarise relevant attributes
    df = df.groupby(by = 'grid_id', as_index=False).agg(lambda x: '; '.join(set(x))).reset_index()
    # convert grid id back to int
    df['grid_id'] = df['grid_id'].astype(int)
    # join columns to grid dataset
    if dset == dsets[0]:
        cols = ['grid_id', 'vernacularName','scientificName']
    else:
        cols = ['grid_id', 'SCI_NAME', 'COM_NAME']
    grid = pd.merge(grid, df[cols], on='grid_id', how='left')
    
grid.rename(columns={'SCI_NAME': 'wildnet_obs_sci_name',
                     'COM_NAME': 'wildnet_obs_com_name',
                     'vernacularName': 'ala_obs_com_name',
                     'scientificName': 'ala_obs_sci_name'},inplace=True)

grid.spatial.to_featureclass('BiodiversityMBRC.gdb/Network_grids/speciesObsGrid')



[10]:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Network_grids\\speciesObsGrid'

In [16]:
## species habitat

##  build species dictionary
evnt = "evntSpeciesTransposed_cleaned"
cols = [field.name for field in arcpy.ListFields(evnt)]
cols.remove('re')
cols.remove('OBJECTID')
print(cols)
res = []
grid = 'Network_grids/speciesHabGrid_Test'

with arcpy.da.SearchCursor(evnt, "re") as cursor:
    for row in cursor:
        if row[0] not in res:
            res.append(row[0])
            
sDict = {key: [] for key in res}

for col in cols:
    arcpy.management.CalculateField(grid, col, '1', 'PYTHON3', '', 'TEXT')
    with arcpy.da.SearchCursor(evnt, ["re", col]) as cursor:
        for row in cursor:
            if row[1] == 1:
                sDict[row[0]].append(col)
                

['swampCrayfish', 'richmondBirdwingButterfly', 'wallumRocketfrog', 'wallumSedgefrog', 'cascadeTreefrog', 'tuskedFrog', 'wallumFroglet', 'commonDeathAdder', 'easternOsprey', 'forkTailedSwift', 'spineTailedSwift', 'beachStoneCurlew', 'glossyBlackCockatoo', 'bandedDotterel', 'greaterSandPlover', 'lesserSandPlover', 'pacificGoldenPlover', 'orientalCuckoo', 'whiteWingedBlackTern', 'gullBilledTern', 'caspianTern', 'commonTern', 'littleTern', 'crestedTern', 'blackFacedMonarch', 'satinFlycatcher', 'spectacledMonarch', 'easternYellowWagtail', 'plumedFrogmouth', 'sootyShearwater', 'swiftParrot', 'rufousFantail', 'australianPaintedSnipe', 'ruddyTurnstone', 'sharpTailedSandpiper', 'redKnot', 'curlewSandpiper', 'redNeckedStint', 'greatKnot', 'lathamsSnipe', 'swinhoesSnipe', 'asianDowitcher', 'westernAlaskanBarTailedGodwit', 'blackTailedGodwit', 'easternCurlew', 'littleWhimbrel', 'whimbrel', 'ruff', 'siberianGreyTailedTattler', 'woodSandpiper', 'commonGreenshank', 'marshSandpiper', 'terekSandpiper',

In [ ]:
for col in cols:
    with arcpy.da.UpdateCursor(grid, ["re", col]) as cursor:
        for row in cursor:
            if row[0]col in sDict[row[0]]:
                row[1] = 'Yes'
            cursor.updateRow(row)

In [6]:
## create council-owned land for land use grid

# merge relevant LGIP datasets

arcpy.management.Merge('Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Civic;Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Rec_and_Foreshore;Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Sports', 
                       'Processing/LGIP_merged',  None, 'NO_SOURCE_INFO')

# pull out council land from MBRC Parcels
arcpy.management.MakeFeatureLayer('Input/MBRCParcels_Registered_Parcels', 'councilParcels', "COUNCIL_LAND IN ('Council Freehold', 'Council reserve', 'Council Leasee')")

# erase MBRC parcels that overlap with merged LGIP datasets
arcpy.analysis.PairwiseErase('councilParcels', 'Processing/LGIP_merged', 'Processing/councilParcels_Erase')

# combine for council owned land dataset
try:
    arcpy.management.Merge('Processing/councilParcels_Erase;Processing/LGIP_merged', 'Input/councilOwnedLand', None, 'NO_SOURCE_INFO')
except:
    arcpy.management.Delete('Input/councilOwnedLand')
    arcpy.management.Merge('Processing/councilParcels_Erase;Processing/LGIP_merged', 'Input/councilOwnedLand', None, 'NO_SOURCE_INFO')




<Result 'councilParcels'>

In [ ]:
######## SUPERSEDED!!! ##########
grid = pd.DataFrame.spatial.from_featureclass("luGrid")

simpleSpatialJoinNear("Input/LGIP_PPCF_Existing_Trunk_Land_for_Community_Facilities", "Processing/luGrid", "Processing/cFJoinGrid", ['grid_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
df = pd.DataFrame.spatial.from_featureclass("cFJoinGrid")
# drop null grid ids
df = df.dropna(subset = 'grid_id')
# convert columns to strings
df = df.astype(str)
# concatenate name and type columns
df['facility'] = df['name'].astype(str) + ' (' + df['type_'].astype(str) + ')'
# group by grid id and summarise relevant attributes
df = df.groupby(by = 'grid_id', as_index=False).agg(lambda x: '; '.join(set(x))).reset_index()
# convert grid id back to int
df['grid_id'] = df['grid_id'].astype(int)
# join columns to grid dataset
grid = pd.merge(grid, df[['facility', 'Closest_Address', 'hierarchy', 'grid_id']], on='grid_id', how='left')
grid.rename(columns={'facility': 'community_facility',
                  'Closest_Address': 'comm_fac_address',
                  'hierarchy': 'comm_fac_hierarchy'},inplace=True)
grid.spatial.to_featureclass("BiodiversityMBRC.gdb/Processing/cFGrid")

# join LGIP area datasets to grid REMOVE THESE!
lgip = {'LGIP_PPCF_Public_Parks_Service_Catchments': 'park',
       'LGIP_PPCF_Future_Trunk_Public_Parks': 'future_park',
       'LGIP_PPCF_Future_Trunk_Land_For_CommunityFacilities': 'future_comm_fac',
       'LGIP_PPCF_Existing_Trunk_Public_Parks_Sports': 'sports_park',
       'LGIP_PPCF_Existing_Trunk_Public_Parks_Rec_and_Foreshore': 'rec_foreshore',
       'LGIP_PPCF_Existing_Trunk_Public_Parks_Civic': 'civic_park'}

i = 0
for dset, prefix in lgip.items():
    arcpy.management.MakeFeatureLayer(f'Input/{dset}', prefix)
    if i == 0:
        inGrid = 'Processing/cFGrid'
    else:
        inGrid = lastGrid
    # define fields to join based on input dataset
    if prefix == 'park':
        fields = ['Service_Catchment']
    elif prefix == 'future_park':
        fields = ['trunk_infra', 'Solution', 'Type', 'est_timing']
    elif prefix == 'future_comm_fac':
        fields = ['trunk_infra', 'Description', 'est_timing']
    else:
        fields = ['Description', 'Type']
    # perform spatial join (largest overlap)
    simpleSpatialJoin(inGrid, prefix, f'Processing/{prefix}_grid', fields, 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    lastGrid = f'{prefix}_grid'
    # rename joined fields with prefix
    for field in fields:
        arcpy.management.AlterField(lastGrid, field, f'{prefix}_{field}', f'{prefix}_{field}')
    i += 1